In [ ]:
%pylab inline
import xarray as xr
import numpy as np
import cartopy
import matplotlib.pyplot as plt
from glob import glob
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

In [ ]:
galapagos_extent = [-91.8+360, -89+360, -1.4, 0.7]

In [ ]:
projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(2, 1, subplot_kw={'projection': projection}, figsize=(8, 8))
for a in ax:
    a.coastlines(resolution='50m')
    a.add_feature(cartopy.feature.LAND)
    a.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                   height=galapagos_extent[3]-galapagos_extent[2], facecolor='r', alpha=0.25, linewidth=1,
                                   edgecolor='k', transform=cartopy.crs.Geodetic(), zorder=0))

# data from ftp://ftp.aoml.noaa.gov/phod/pub/buoydata/hourly_product/v1.02/
files = sorted(glob('/Volumes/oceanparcels/input_data/DriftingBuoys/*.nc'))
labels = ['Drifter sections with drogue', 'Drifter sections w/o drogue', 'Drifter sections with drogue', 'Drifter sections w/o drogue']
drifters_in = 0
drifters_out = 0
drifters_labels = [0, 0, 0, 0]
totdrifters = 0
alldrifters = 0
for fi in files:
    D = xr.open_dataset(fi)
    Idrogue = np.where(D['DROGUE'] > 0.5)
    Iundrogue = np.where(D['DROGUE'] < 0.5)
    
    gidlat = np.where(np.logical_and(D['LAT'] >= galapagos_extent[2], D['LAT'] <= galapagos_extent[3]))
    gidlon = np.where(np.logical_and(D['LON'] >= galapagos_extent[0], D['LON'] <= galapagos_extent[1]))
    gidlonlat = np.intersect1d(gidlon, gidlat)
    gids = np.unique(D['ID'][gidlonlat])
    print(fi, len(gids))
    totdrifters += len(gids)
    alldrifters += len(np.unique(D['ID']))

    for gi in gids:
        I = np.where(D['ID'] == gi)[0]
        lon = D['LON'][I]
        lat = D['LAT'][I]
        drogue = D['DROGUE'][I]
        idlat = np.where(np.logical_and(lat >= galapagos_extent[2], lat <= galapagos_extent[3]))
        idlon = np.where(np.logical_and(lon >= galapagos_extent[0], lon <= galapagos_extent[1]))
        idlonlat = np.intersect1d(idlon, idlat)
        
        if idlonlat[0] > 24:
            drifters_in += 1
        if idlonlat[-1] < len(lon) - 24:
            drifters_out += 1

        II = np.intersect1d(np.where(drogue > 0.5), range(idlonlat[0]))
        if len(II) > 0:
            drifters_labels[0] += 1
            ax[0].plot(lon[II], lat[II], 'k', transform=cartopy.crs.Geodetic(), label=labels[0])
            labels[0] = '_nolegend_'
        II = np.intersect1d(np.where(drogue < 0.5), range(idlonlat[0]))
        if len(II) > 0:
            drifters_labels[1] += 1
            ax[0].plot(lon[II], lat[II], 'b', transform=cartopy.crs.Geodetic(), label=labels[1])
            labels[1] = '_nolegend_'

        II = np.intersect1d(np.where(drogue > 0.5), range(idlonlat[-1], len(lon)))
        if len(II) > 0:
            drifters_labels[2] += 1
            ax[1].plot(lon[II], lat[II], 'k', transform=cartopy.crs.Geodetic(), label=labels[2])
            labels[2] = '_nolegend_'
        II = np.intersect1d(np.where(drogue < 0.5), range(idlonlat[-1], len(lon)))
        if len(II) > 0:
            drifters_labels[3] += 1
            ax[1].plot(lon[II], lat[II], 'b', transform=cartopy.crs.Geodetic(), label=labels[3])
            labels[3] = '_nolegend_'

print('Total number of drifters is %d of %d' % (totdrifters, alldrifters))
print('Drifters in %d' % drifters_in)
print('Drifters out %d' % drifters_out)

L = ax[0].legend()
L.get_texts()[0].set_text('Drifter sections with drogue (n=%d)' % drifters_labels[0])
L.get_texts()[1].set_text('Drifter sections w/o drogue (n=%d)' % drifters_labels[1])
ax[0].set_title('Drifters arriving in Galapagos region')
L = ax[1].legend()
L.get_texts()[0].set_text('Drifter sections with drogue (n=%d)' % drifters_labels[2])
L.get_texts()[1].set_text('Drifter sections w/o drogue (n=%d)' % drifters_labels[3])
ax[1].set_title('Drifters leaving Galapagos region')
plt.tight_layout()
savefig('buoys.pdf')
